In [1]:
from operator import itemgetter
from ipywidgets import IntProgress, HBox, Label, Layout
from IPython.display import display, HTML
import os
print(os.getcwd())

/Users/jiricerny/_Git/_venv/_venv/Python_2024/files


In [2]:
SUFFIXES = {1000:['KB','MB','GB','TB','PB','ZB','YB'],
            1024:['KiB','MiB','GiB','TiB','PiB','ZiB','YiB']}

def approximate_size(size, a_kilobite_is_1024_bytes=True):
    '''Convert

    Returns: string
    '''
    if size < 0:
        raise ValueError('nubmer must be non-negative')

    multiple = 1024 if a_kilobite_is_1024_bytes else 1000
    for suffix in SUFFIXES[multiple]:
        size /= multiple
        if size < multiple:
            return '{0:.1f} {1}'.format(size, suffix)

    raise ValueError('nubmer too large')

In [3]:
def print_all (path):
    global files_all,size_all,path_list,path_dict,files_list,files_dict,extentions_list,extentions_dict,superpath_list,superpath_dict, iBar
    #dirs = [(os.path.realpath(entry.name)) for entry in os.scandir(path) if not entry.name.startswith('.') and entry.is_dir()]
    dirs = [(entry.path) for entry in os.scandir(path) if not entry.name.startswith('.') and entry.is_dir()]
    #print (dirs)
    # print(path, len(dirs)) '''vypíše cestu a počet podadresářů; pokud > 0 tak má podadresáře => zapíše se jako superpath'''
    if len (dirs) > 0:
        superpath_list.append(path)
        superpath_dict[path] = [0,0,-1] #jinak by to i aktuální adresář napočítalo jako +1
    
    iBar.max += len (dirs)
   
    size_path = 0

    #files = [(os.path.realpath(entry.name)) for entry in os.scandir(path) if not entry.name.startswith('.') and entry.is_file()]
    files = [(entry.path) for entry in os.scandir(path) if not entry.name.startswith('.') and entry.is_file()]
    #print (files)

    metadata_dict = {os.path.realpath(f):os.stat(f) for f in files}

    for f in range(len(files)):
        #print (files[f],approximate_size(metadata_dict[files[f]].st_size,False))
        files_list.append(os.path.split(files[f]))
        files_dict[os.path.realpath(files[f])]=metadata_dict[files[f]]
        size = metadata_dict[files[f]].st_size
        size_path += size
        extention = os.path.splitext(files[f])[1]
        i=0
        for e in extentions_list:
            if extention == e: i = 1
        if i == 0:
            extentions_list.append(extention)
            extentions_dict[extention] = [0,0]
        extentions_dict[extention][0] += 1
        extentions_dict[extention][1] += size

    #print (len (files), approximate_size(size,False)) - toto se vypisuje
    files_all += len (files)
    size_all += size_path
    path_list.append(path)
    path_dict[path]=[len(files),size_path, len(dirs)]
    for sp in superpath_list:
        if sp == path[0:len(sp)]:
            superpath_dict[sp][0] += len (files)
            superpath_dict[sp][1] += size_path
            superpath_dict[sp][2] += 1
            '''toto napočítá i podpodadresáře!!!'''
    
    for d in range(len (dirs)):
        print_all (dirs[d])
        iBar.value += 1
        label.value = str(iBar.value) + "/" + str(iBar.max)

In [4]:
iBar = IntProgress()
iBar.value = 0
iBar.max = 0
label = Label("")
labels =HBox(children=[Label("AHOJ"), iBar, label], layout=Layout(width='700px', margin='0px 0px 0px 35px'))
display(labels)

files_all=0
size_all=0
path_list=[]
path_dict={}
files_list=[]
files_dict={}
extentions_list=[]
extentions_dict={}
superpath_list=[]
superpath_dict={}
#print_all('C:\LOCAL_DATA')
#print_all('/Users/jiricerny/Downloads')
print_all('/Volumes/Transcend/# Fotky')

In [5]:
temp_p = []
for e in path_dict:
    temp_p.append([e,str.lower(e),path_dict[e][0],path_dict[e][1],path_dict[e][2]])
'''print ()
temp_p.sort(key=itemgetter(1))
for e in range(len(temp_p)):
    print (temp_p[e][0],temp_p[e][1],temp_p[e][2],temp_p[e][3])'''
print ()
print ('Seřaď podle názvu:')
temp_p.sort(key=itemgetter(1))
for e in temp_p:
    print (e[0],path_dict[e[0]][0],approximate_size(path_dict[e[0]][1],False),path_dict[e[0]][2])
print ()
print ('Seřaď podle počtu souborů:')
temp_p.sort(key=itemgetter(2))
for e in temp_p:
    print (e[0],path_dict[e[0]][0],approximate_size(path_dict[e[0]][1],False),path_dict[e[0]][2])
print ()
print ('Seřaď podle velikosti:')
temp_p.sort(key=itemgetter(3))
for e in temp_p:
    print (e[0],path_dict[e[0]][0],approximate_size(path_dict[e[0]][1],False),path_dict[e[0]][2])
print ()
print ('Seřaď podle počtu adresářů:')
temp_p.sort(key=itemgetter(4))
for e in temp_p:
    print (e[0],path_dict[e[0]][0],approximate_size(path_dict[e[0]][1],False),path_dict[e[0]][2])

print ()
for sp in sorted(superpath_list,key=str.lower):
    print (sp,superpath_dict[sp][0],approximate_size(superpath_dict[sp][1],False),superpath_dict[sp][2])

temp_e = []
for e in extentions_dict:
    temp_e.append([e,str.lower(e),extentions_dict[e][0],extentions_dict[e][1]])
'''print ()
temp_e.sort(key=itemgetter(1))
for e in range(len(temp_e)):
    print (temp_e[e][0],temp_e[e][1],temp_e[e][2],temp_e[e][3])'''
print ()
temp_e.sort(key=itemgetter(1))
for e in temp_e:
    print (e[0],extentions_dict[e[0]][0],approximate_size(extentions_dict[e[0]][1],False))
print ()
temp_e.sort(key=itemgetter(2))
for e in temp_e:
    print (e[0],extentions_dict[e[0]][0],approximate_size(extentions_dict[e[0]][1],False))
print ()
temp_e.sort(key=itemgetter(3))
for e in temp_e:
    print (e[0],extentions_dict[e[0]][0],approximate_size(extentions_dict[e[0]][1],False))
'''for e in sorted(extentions_dict,key=itemgetter(2)):
    print (e,extentions_dict[e][0],extentions_dict[e][1])
    #print (e,extentions_dict[e][0],approximate_size(extentions_dict[e][1],False))'''

print ()
print ('Konec...',files_all,approximate_size(size_all,False))
#print ('Konec...',files_all,approximate_size(size_all,False),approximate_size(size_all/files_all,False)) - vypíše i průměr


Seřaď podle názvu:
/Volumes/Transcend/# Fotky 0 0.0 KB 14
/Volumes/Transcend/# Fotky/2013_07_06-13 Ostružná 518 5.7 GB 0
/Volumes/Transcend/# Fotky/2013_07_15-19 Zbraslavice 63 1.1 GB 0
/Volumes/Transcend/# Fotky/2013_08_25-30 Mariánské Lázně 431 6.0 GB 0
/Volumes/Transcend/# Fotky/2014_05_07-10 Budapešť 383 4.1 GB 0
/Volumes/Transcend/# Fotky/2014_07_21-26 267 3.8 GB 0
/Volumes/Transcend/# Fotky/2014_08_25-29 384 4.5 GB 0
/Volumes/Transcend/# Fotky/2015_07_19-25 665 7.4 GB 0
/Volumes/Transcend/# Fotky/20160423-24 Sušice 109 1.2 GB 0
/Volumes/Transcend/# Fotky/20180330 Legoland 793 5.6 GB 0
/Volumes/Transcend/# Fotky/2019_07_30-08_08 650 4.6 GB 0
/Volumes/Transcend/# Fotky/Krakow 66 701.8 MB 0
/Volumes/Transcend/# Fotky/Norsko 2036 9.9 GB 0
/Volumes/Transcend/# Fotky/Norsko_MS 13 4.4 MB 0
/Volumes/Transcend/# Fotky/Norsko_vyber 219 906.8 MB 0

Seřaď podle počtu souborů:
/Volumes/Transcend/# Fotky 0 0.0 KB 14
/Volumes/Transcend/# Fotky/Norsko_MS 13 4.4 MB 0
/Volumes/Transcend/